In [2]:
import numpy as np
from matplotlib import pyplot as plt
from math import pi as pi
from scipy.spatial.transform import Rotation as R
import tensorflow
from tensorflow.keras import mixed_precision


policy = mixed_precision.Policy('float32')
mixed_precision.set_global_policy(policy)

size = 16


# Probably need to rewrite this but use v0 as the random variable
def get_rand_tetra_loc(num_tetra, v0, v1, v2, v3, Flip = False):
   
    trans = np.zeros((3, num_tetra))
    trans[0,:] = np.random.uniform(-5.,5., num_tetra)
    trans[1,:] = np.random.uniform(-5.,5., num_tetra)
    trans[2,:] = np.random.uniform(-5.,5., num_tetra)

    rot_arr = R.random(num = num_tetra).as_matrix()
   
    if Flip == True:
        #flip_idx = np.random.choice(num_tetra, int(num_tetra/2), replace=False)
        v0_flip, v1_flip, v2_flip, v3_flip = -v0.copy(), -v1.copy(), -v2.copy(), -v3.copy()
       
    tetra = np.zeros((4,3,num_tetra))
    for i in range(num_tetra):
        if Flip == True:
            tetra[0,:,i] = np.dot(rot_arr[i,:,:], v0_flip) + trans[:,i]
            tetra[1,:,i] = np.dot(rot_arr[i,:,:], v1_flip) + trans[:,i]
            tetra[2,:,i] = np.dot(rot_arr[i,:,:], v2_flip) + trans[:,i]
            tetra[3,:,i] = np.dot(rot_arr[i,:,:], v3_flip) + trans[:,i]

        else:
            tetra[0,:,i] = np.dot(rot_arr[i,:,:], v0) + trans[:,i]
            tetra[1,:,i] = np.dot(rot_arr[i,:,:], v1) + trans[:,i]
            tetra[2,:,i] = np.dot(rot_arr[i,:,:], v2) + trans[:,i]
            tetra[3,:,i] = np.dot(rot_arr[i,:,:], v3) + trans[:,i]
               
       
    return tetra



num_left_tet = 3
num_right_tet = 1
v0 = np.array([8,0,0])
v1 = np.array([0,4,0])
v2 = np.array([0,0,2])
v3 = np.array([0,0,0])


def get_rand_tet_excess(num_left_tet, num_right_tet, v0, v1, v2, v3, master_flip = False):
    n_tet = num_left_tet + num_right_tet
    dat = np.zeros((4,3,n_tet))
    if master_flip == False:
        dat[:,:,:num_left_tet] = get_rand_tetra_loc(num_left_tet, v0, v1, v2, v3, Flip = False)
        dat[:,:,num_left_tet:] = get_rand_tetra_loc(num_right_tet, v0, v1, v2, v3, Flip = True)
    else:
        dat[:,:,:num_left_tet] = get_rand_tetra_loc(num_left_tet, v0, v1, v2, v3, Flip = True)
        dat[:,:,num_left_tet:] = get_rand_tetra_loc(num_right_tet, v0, v1, v2, v3, Flip = False)      
    return dat



def get_ran_field(tetra):
   
    num_tetra = np.shape(tetra)[2]
    verts = np.zeros((4 * num_tetra, 3))
    for v in range(4):
        for n in range(num_tetra):
            verts[4*n+v,:] = tetra[v,:,n]

    npix = size
    mn = -size
    mx = size
    pix_sz = (mx - mn) / npix

    grid = np.zeros((size,size,size))      
    for v in range(4 * num_tetra):
        x = verts[v,0]
        y = verts[v,1]
        z = verts[v,2]
        xpix = int((x - mn) / pix_sz )
        ypix = int((y - mn) / pix_sz )
        zpix = int((z - mn) / pix_sz )
        grid[xpix, ypix, zpix] += 1
    return grid


test = get_ran_field(get_rand_tet_excess(num_left_tet, num_right_tet, v0, v1, v2, v3)) 


from scatter import Scatter3D

J = 4
L = 4
M = 16
slant1 = 4./L
slant2 = 2./L

ntrain = 1000
nvalid = 100

nbatch_train = 20
nbatch_valid = 10
batch_size_train = int(ntrain / nbatch_train)
batch_size_valid = int(nvalid / nbatch_valid)


xtrain, xvalid = np.zeros((batch_size_train,size,size,size)), np.zeros((batch_size_valid,size,size,size))


inp_train = (size, size, size)
inp_valid = inp_train

Scat = Scatter3D(J,L,M,slant1, slant2)
filter_bank = Scat.filter_bank()



x = tensorflow.ones((10,16,16,16))


phi = filter_bank['phi']
psi = filter_bank['psi']
max_order = 3
coeffs = Scat.compute_coefs_no_pad_extend(x, phi, psi, max_order, out_type = 'array')
print (np.shape(coeffs))

compressed_shape = np.shape(coeffs)[2]


Xtrain, Xtrain_flip = np.zeros((ntrain,compressed_shape)), np.zeros((ntrain,compressed_shape)) 
for i in range(nbatch_train):
    print (i)
    for j in range(batch_size_train):
        xtrain[j,:,:,:] = get_ran_field(get_rand_tet_excess(num_left_tet, num_right_tet, v0, v1, v2, v3))
    xtrain_flip = xtrain[:,::-1,:,:]
    Xtrain[i*batch_size_train:(i+1)*batch_size_train] =\
    Scat.compute_coefs_no_pad_extend(xtrain, phi, psi, max_order, out_type = 'array')[:,0,:,0,0]
    Xtrain_flip[i*batch_size_train:(i+1)*batch_size_train] =\
    Scat.compute_coefs_no_pad_extend(xtrain_flip, phi, psi, max_order, out_type = 'array')[:,0,:,0,0]
    
    
Xvalid, Xvalid_flip = np.zeros((nvalid,compressed_shape)), np.zeros((nvalid,compressed_shape)) 
for i in range(nbatch_valid):
    print (i)
    for j in range(batch_size_valid):
        xvalid[j,:,:,:] = get_ran_field(get_rand_tet_excess(num_left_tet, num_right_tet, v0, v1, v2, v3))
    xvalid_flip = xvalid[:,::-1,:,:]
    Xvalid[i*batch_size_valid:(i+1)*batch_size_valid] =\
    Scat.compute_coefs_no_pad_extend(xvalid, phi, psi, max_order, out_type = 'array')[:,0,:,0,0]
    Xvalid_flip[i*batch_size_valid:(i+1)*batch_size_valid] =\
    Scat.compute_coefs_no_pad_extend(xvalid_flip, phi, psi, max_order, out_type = 'array')[:,0,:,0,0]
    


(10, 1, 58945, 1, 1)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9


In [6]:
s1 = np.shape(Xtrain)[1]
from tensorflow.keras.layers import Dense, Input, Subtract
from keras.models import Model

A1 = Input(shape=(s1), name='input layer 1')
B1 = Input(shape=(s1), name='input layer 2')


d1 = Dense(100, activation = 'relu')
A2 = d1(A1)
B2 = d1(B1)


d2 = Dense(100, activation = 'relu')
A4 = d2(A2)
B4 = d2(B2)



output = Dense(1, activation = 'linear', name = 'Output')
A6 =  output(A4)
B6 =  output(B4)  


out = Subtract()([A6,B6])

merged3 = Model(inputs=[A1, B1],outputs=[out])

In [7]:
class CustomLoss(tensorflow.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):        
        return -tensorflow.math.reduce_mean(y_pred) / tensorflow.math.reduce_std(y_pred)

In [8]:
ytrain = np.zeros(np.shape(Xtrain)[0])
yvalid = np.zeros(np.shape(Xvalid)[0])



adamOpti = tensorflow.keras.optimizers.Adam (learning_rate = 0.001)
merged3.compile(optimizer = adamOpti, loss = CustomLoss())

history = merged3.fit(x = [Xtrain*1e-7, Xtrain_flip*1e-7], y = ytrain, \
                    validation_data=([Xvalid*1e-7, Xvalid_flip*1e-7], yvalid), epochs = 100, batch_size = 64)

Epoch 1/100
16/16 [==============================] - 1s 39ms/step - loss: -4.1951e-04 - val_loss: 0.0903
Epoch 2/100
16/16 [==============================] - 1s 35ms/step - loss: -0.0395 - val_loss: 0.1112
Epoch 3/100
16/16 [==============================] - 1s 35ms/step - loss: -0.0557 - val_loss: 0.1407
Epoch 4/100
16/16 [==============================] - 1s 37ms/step - loss: -0.0610 - val_loss: 0.1492
Epoch 5/100
16/16 [==============================] - 1s 38ms/step - loss: -0.0492 - val_loss: 0.1625
Epoch 6/100
16/16 [==============================] - 1s 37ms/step - loss: -0.0745 - val_loss: 0.1684
Epoch 7/100
16/16 [==============================] - 1s 40ms/step - loss: -0.0744 - val_loss: 0.1818
Epoch 8/100
16/16 [==============================] - 1s 39ms/step - loss: -0.0837 - val_loss: 0.1718
Epoch 9/100
16/16 [==============================] - 1s 40ms/step - loss: -0.0786 - val_loss: 0.1604
Epoch 10/100
16/16 [==============================] - 1s 41ms/step - loss: -0.0728 - va